In [1]:
import pickle
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Lambda

Using TensorFlow backend.


In [2]:
data_dict = pickle.load(open('../data/alice/processed-alice.pickle', 'rb'))

In [3]:
X = data_dict['X']
Y = data_dict['Y']
dataX = data_dict['dataX']
dataY = data_dict['dataY']
char_to_idx = data_dict['char_to_idx']
idx_to_char = data_dict['idx_to_char']

# Constants
m, n_timesteps, _ = X.shape
_, n_chars = Y.shape
n_a = 64

In [4]:
print('X shape:', X.shape)
print('Y shape:', Y.shape)
print('Num. Timesteps:', n_timesteps)
print('Num. Unique Chars:', n_chars)

X shape: (144224, 100, 1)
Y shape: (144224, 45)
Num. Timesteps: 100
Num. Unique Chars: 45


In [22]:
def get_model(n_timesteps, n_features, n_a):
    x0 = Input(shape=(n_timesteps,1), name='input')
    
    X = LSTM(n_a, return_sequences=True)(x0)
    X = LSTM(n_a)(X)
    out = Dense(n_features, activation='softmax')(X)
    model = Model(x0, out)
    
    return model

In [23]:
model = get_model(n_timesteps, n_chars, n_a)

In [24]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [25]:
a_initial = np.zeros((n_timesteps, n_a))
c_initial = np.zeros((n_timesteps, n_a))

In [35]:
model.fit(X, Y, epochs=25, batch_size=256)

Epoch 1/25
144224/144224 [==============================] - 90s 621us/step - loss: 2.2912
Epoch 2/25
144224/144224 [==============================] - 84s 581us/step - loss: 2.2690
Epoch 3/25
144224/144224 [==============================] - 82s 570us/step - loss: 2.2472
Epoch 4/25
144224/144224 [==============================] - 82s 571us/step - loss: 2.2311
Epoch 5/25
144224/144224 [==============================] - 88s 607us/step - loss: 2.2147
Epoch 6/25
144224/144224 [==============================] - 89s 614us/step - loss: 2.1971
Epoch 7/25
144224/144224 [==============================] - 82s 570us/step - loss: 2.1823
Epoch 8/25
144224/144224 [==============================] - 83s 574us/step - loss: 2.1695
Epoch 9/25
144224/144224 [==============================] - 83s 572us/step - loss: 2.1553
Epoch 10/25
144224/144224 [==============================] - 82s 571us/step - loss: 2.1429
Epoch 11/25
144224/144224 [==============================] - 82s 570us/step - loss: 2.1316
Epoch 12

In [36]:
total_epochs = 50
model.save('../data/trained-{}-epochs.h5'.format(total_epochs))

In [37]:
# Generate Text
# Given seed sentence - 100 characters - predict next character
# For every predicted char, add to predictions = []

def reshape_input(original_input):
    return np.reshape(original_input, (1, n_timesteps, 1))

def get_p_idx(p):
    flattened = np.ndarray.flatten(np.array(p))
    return np.random.choice([i for i in range(n_chars)], 
                            p = flattened)

def generate_text(seed_input, model, num_chars_to_generate=140):
    if len(seed_input) < 100:
        raise Exception('Seed_input must be at least 100 characters')
    curr_input = seed_input.lower()
    curr_input = list(curr_input)
    curr_input = curr_input[:100] # first 100 chars
    
    curr_input = [char_to_idx[c] for c in curr_input]
    # Normalize
    curr_input = np.array(curr_input) / n_chars
    predictions = []
    
    for i in range(num_chars_to_generate):
        p = model.predict(reshape_input(curr_input))
        p_idx = get_p_idx(p)
        predictions.append(idx_to_char[p_idx])
        curr_input = np.append(curr_input[1:], p_idx / n_chars)
    
    return ''.join(predictions)

In [38]:
seed_input = "its a race its a race! Lets put it to the test and be our very best. Its a race! Oh my Oh my I must really try to get more characters."

In [39]:
generate_text(seed_input, model)

'ar agpi  \nguesiin to get tellgt would to soto!ouoted.-\n\naolwgouse nf thmugdr the huomd\nhage\n cos the hard bnitinnsteidn the\nvrlamd. buthdns '